In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from adaptive_sampling.processing_tools import mbar, thermodynamic_integration

In [ ]:
traj_dat = np.loadtxt('CV_traj.dat', skiprows=1)

In [ ]:
ext_sigma = 2.0    # thermal width of coupling between CV and extended variable 

# grid for free energy profile can be different than during sampling
minimum   = 70.0     
maximum   = 170.0    
bin_width = 2.0    
grid = np.arange(minimum, maximum, bin_width)

cv = traj_dat[:,1]  # trajectory of collective variable
la = traj_dat[:,2]  # trajectory of extended system

In [ ]:
# run MBAR and compute free energy profile and probability density from statistical weights
traj_list, indices, meta_f = mbar.get_windows(grid, cv, la, ext_sigma, equil_temp=300.0)
weights = mbar.run_mbar(traj_list, meta_f, conv=1.0e-4, conv_errvec=None, outfreq=100, equil_temp=300.0)

In [ ]:
pmf, rho = mbar.pmf_from_weights(grid, cv[indices], weights, equil_temp=300.0)

In [ ]:
# czar free energy estimate
czar_force = thermodynamic_integration.czar(grid, cv, la, ext_sigma, equil_temp=300.0)
pmf_ti, rho_ti = thermodynamic_integration.integrate(czar_force, bin_width, equil_temp=300.0)

In [ ]:
fig, axs = plt.subplots(1, figsize=(8,6))

axs.plot(grid, pmf, linewidth=4, label=r'eABF/MBAR')
axs.plot(grid+bin_width, pmf_ti, linewidth=4, label='eABF/CZAR')

axs.tick_params(axis='y',length=8,width=4,labelsize=30, pad=10, direction='in')
axs.tick_params(axis='x',length=8,width=4,labelsize=30, pad=10, direction='in')
#axs.grid()
axs.legend(fontsize=25, loc='best', frameon=False, shadow=False)
axs.set_xlabel(r'$x$', fontsize=40)
axs.set_ylabel(r'$A(x)$ / kJ mol$^{-1}$', fontsize=40)
axs.spines['bottom'].set_linewidth('3')
axs.spines['top'].set_linewidth('3')
axs.spines['left'].set_linewidth('3')
axs.spines['right'].set_linewidth('3')

fig.tight_layout()